<a href="https://colab.research.google.com/github/LC1332/Needy-Haruhi/blob/main/notebook/%E4%BB%8E%E8%A1%A8%E6%A0%BC%E4%B8%AD%E8%A7%A3%E6%9E%90%E4%BA%8B%E4%BB%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 下载文件。
!wget https://github.com/LC1332/Needy-Haruhi/raw/main/data/Jines.csv

--2023-11-05 02:16:18--  https://github.com/LC1332/Needy-Haruhi/raw/main/data/Jines.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LC1332/Needy-Haruhi/main/data/Jines.csv [following]
--2023-11-05 02:16:18--  https://raw.githubusercontent.com/LC1332/Needy-Haruhi/main/data/Jines.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249489 (244K) [text/plain]
Saving to: ‘Jines.csv’

Jines.csv           100%[===================>] 243.64K  --.-KB/s    in 0.03s   

2023-11-05 02:16:19 (9.50 MB/s) - ‘Jines.csv’ saved [249489/249489]



In [2]:
import os
import re
import shutil

import pandas as pd

Jines_file = r"/content/Jines.csv"  # 更改文件路径和扩展名

# # 读取CSV文件
Jines = pd.read_csv(Jines_file, lineterminator='\n')

In [4]:
import os
import re
import shutil

import pandas as pd

Jines_file = r"/content/Jines.csv"  # Change the file path and extension as needed

# Read the CSV file
Jines = pd.read_csv(Jines_file, lineterminator='\n')




实现一个python函数，输入是字符串，输出也是一个字符串，找到第一个"("，取括号之前的字符串并strip后输出

例子输入
Day0_JINE (First Part)
例子输出
Day0_JINE

In [15]:
def extract_string_before_parentheses(input_value):
    # Check if the input is not a string
    if pd.isnull(input_value):
        return ""
    elif not isinstance(input_value, str):
        print("Warning: Input is not a string. Converting to string.")
        # Convert to string if possible
        input_value = str(input_value)
        print(input_value)

    # Split the string at the first occurrence of '('
    parts = input_value.split('(', 1)
    # Take the first part and strip it of whitespace
    return parts[0].strip()

# Example usage with a string:
input_example_str = "Day0_JINE (First Part)"
print(extract_string_before_parentheses(input_example_str))  # Expected output: Day0_JINE

# Example usage with a non-string:
input_example_non_str = 12345
print(extract_string_before_parentheses(input_example_non_str))  # Expected output: Warning + "12345"


Day0_JINE
12345
12345


In [28]:
count = 0

lines = []

last_parent = ""
last_category = ""

all_events = []

# Loop through each row and print the 'Category' and 'Parent'
for index, row in Jines.iterrows():

    if pd.isnull(row['ParentId (more info)']):
        continue

    parent = extract_string_before_parentheses(row['ParentId (more info)'])
    category = row['Category']

    if parent.startswith("Ending"):
        break


    # print(f"Category: {category}, Parent: {parent}")

    if category == last_category and last_parent == parent:
        lines.append(row)
    else:
        data = {
            "parent": last_parent,
            "category": last_category,
            "lines": lines
        }
        all_events.append(data)

        last_parent = parent
        last_category = category
        lines = [row]

if len(lines) > 0:
    data = {
        "parent": last_parent,
        "category": last_category,
        "lines": lines
    }
    all_events.append(data)

all_events = all_events[1:]

In [29]:
print(len(all_events))

196


In [ ]:
for i, event in enumerate(all_events):
    if i % 3 == 0:
        print(i, event['parent'])
    else:
        print(i, event['parent'], end = ' ')

In [ ]:
for i, event in enumerate(all_events):
    only_ame_flag = True
    for line in event['lines']:
        speaker = line["Speaker/Action (in blue)"]
        if speaker != "ame":
            only_ame_flag = False
            break
    if only_ame_flag:
        print(event['parent'])

In [79]:
def check_only_ame(event):
    for line in event['lines']:
        speaker = line["Speaker/Action (in blue)"]
        if speaker == "pi":
            return False
    return True

def check_2nd(event):
    for line in event['lines']:
        parent = line['ParentId (more info)']
        if "2ndOption" in parent:
            return True
    return False

In [80]:
def check_open(event):
    for line in event['lines']:
        speaker = line["Speaker/Action (in blue)"]
        if speaker == "Any Open-Text Answer":
            return True
    return False

In [81]:
def transfer_dialogue(event):
    dialogues = []

    last_speaker = ""
    last_text = ""
    last_title = ""
    for i, line in enumerate(event['lines']):
        speaker = line["Speaker/Action (in blue)"]
        title = line["Id"]
        text = line["BodyCn"]
        if speaker != last_speaker or i + 1 == len(event['lines']):
            if last_speaker != "":
                dialogues.append({
                    "speaker": last_speaker,
                    "text": last_text,
                    "title": last_title
                })
            last_speaker = speaker
            last_text = text
            last_title = title
        else:
            last_text += text + " "

    return dialogues

如果是一开始的ame的部分，直接作为prefix

然后pi的时候要看相同的parent

然后后面如果有不一样的就作为post


In [111]:
count = 0
for i, event in enumerate(all_events):
    if check_only_ame(event):
        continue

    if check_2nd(event):
        continue

    if check_open(event):
        continue

    parent = event['parent']
    # print(parent, ' ' , event['category'],' ', len(event['lines']))

    # dealing with one event
    state = "count_prefix"

    prefix = ""
    options = []

    # if parent in ["Day0_JINE"]:
    #     verbose = True
    # else:
    #     verbose = False

    verbose = False

    record_flag = True

    for i, line in enumerate(event['lines']):

        speaker = line["Speaker/Action (in blue)"]
        line_parent = line['ParentId (more info)']
        content = line["BodyCn"]

        if verbose:
            print( speaker ," ", line_parent, " ", content, " ", len(options))


        if i != len(event['lines']) - 1:
            next_parent = event['lines'][i + 1]['ParentId (more info)']
            next_parent = next_parent.replace(";end", "")
            next_speaker = event['lines'][i + 1]['Speaker/Action (in blue)']
        else:
            next_parent = ""
            next_speaker = ""

        if state == "count_prefix":
            if verbose:
                print("state = ", state )

            if speaker == "ame":
                prefix += line["BodyCn"] + "\n"
                continue
            elif speaker == "pi":
                state = "count_option_start"

        if state == "count_option_start":
            if verbose:
                print("prefix = ")
                print(prefix)
                print("state = ", state )

            if speaker == "pi":
                option_text = line["BodyCn"]
                state = "collect_reply"
                option_parent = line_parent
                # print("opt_parent=", option_parent)
                # print("next_parent=", next_parent)
                reply = ""

                if next_speaker == "pi":
                    # 说明下一个也是选项，要直接终结掉这个选项
                    state = "count_post"
                else:
                    continue
            else:
                print("warning! not pi's reply in count_option_start, Event = ", parent)

        if state == "collect_reply":
            if verbose:
                print("state = ", state )

            # if speaker != "ame" and speaker != "pi":
            #     # print("skip speaker ", speaker)
            #     continue

            if speaker == "ame":
                option_text += line["BodyCn"]
            elif speaker == "pi":
                # a new reply
                print("warning! not ame's reply in collect_reply, Event = ", parent)

            if i == len(event['lines']) - 1:
                state = "count_post"
            else:
                if next_parent != option_parent:
                    state = "count_post"

        if state == "count_post":
            if verbose:
                print("state = ", state )
            option_data = {
                "text": option_text,
                "reply": reply
            }

            options.append(option_data)

            if i == len(event['lines']) - 1:
                break

            next_speaker = event['lines'][i + 1]['Speaker/Action (in blue)']

            if "Option" in next_parent and next_speaker == "pi":
                state = "count_option_start"
            elif "Option" not in next_parent and next_speaker == "ame":
                state = "collect_post"
            else:
                # print("strange", next_parent, ' ', next_speaker, ' ', content)
                record_flag = False


            continue

    if record_flag:
        event_name = extract_string_before_parentheses(parent)
        print(event_name, " len_prefix = " , len(prefix), " #opt = ", len(options))
        print(prefix)

        count += 1


        # break

        # if state == "count_option_end":
        #     if speaker == "ame":
        #         option_text += line["BodyCn"]
        #     else:
        #         print("warning! not ame's reply in count_option_end")

    # print("prefix:\n",prefix)
    # print("first option:\n",option_text)
    # print()
    # count += 1
    # if count > 5:
    #     break


print(count)

Event_UberEats  len_prefix =  25  #opt =  3
我们点外卖吧我一步也不想动了可是又超想吃饭！！！

Event_Sea  len_prefix =  27  #opt =  2
我已经彻底疲倦了
不如我们结束这一切 现在就去海边吧

Event_Pudding  len_prefix =  26  #opt =  3
我没打招呼就把冰箱里的布丁吃了 会被判死刑吗？？？

Event_Hairstyle  len_prefix =  20  #opt =  4
想换个发型了，阿P喜欢什么样子的糖糖？

Event_Money  len_prefix =  15  #opt =  3
我要出去玩！给我零花钱！！！

Event_Seikei  len_prefix =  18  #opt =  3
如果我要整容，你觉得整哪里比较好？

Event_AmePiercerd  len_prefix =  70  #opt =  2
嗳，你来帮我打耳洞嘛 让喜欢的人给自己打耳洞很棒不是吗 有一种被支配着的感觉 鸡皮疙瘩都要起来了
我好怕我好怕我好怕
我好怕！
但是来吧！

Event_Charahen  len_prefix =  14  #opt =  3
哎，你喜欢什么样的糖糖啊？

Event_AmeFuture  len_prefix =  18  #opt =  3
哎，你会希望看到糖糖将来的样子吗？

Event_Sumabura  len_prefix =  41  #opt =  2
我也想被做进那个大乱斗游戏……
哎，如果那个游戏里面有超天酱的话，阿P会用我吗？

Event_Negativ  len_prefix =  37  #opt =  3
光是活着就好累啊……
现在无论是谁对我说什么，我肯定都会往负面方向去理解

Event_DrugHolic  len_prefix =  46  #opt =  2
啊～不行了 不行不行不行不行
无论思考什么，满脑袋都只有一个“死”字
阿P，我该怎么办啊？

Event_Jisatumisui  len_prefix =  30  #opt =  3
不行了 我现在就想立刻马上消失
阿P 我们一起去买炭吧……

Event_Flower  len_prefix =  12 

In [33]:

for event in all_events:
    flag = False
    for line in event['lines']:
        parent = line['ParentId (more info)']
        if "2ndOption" in parent:
            flag = True
            break

    if flag:
        print(event['parent'])

Event_NextDate
Event_Yandeiru
Event_Yutabon
Event_MailInterview
Event_Dialog


In [35]:
count = 0

for event in all_events:
    if len(event['lines']) == 1:
        continue

    flag = False
    for line in event['lines']:
        parent = line['ParentId (more info)']
        if "2ndOption" in parent:
            flag = True
            break

    if flag:
        continue

    count += 1

print(count)

176


In [61]:



# # 查询有选项的内容
pattern = r'(?<=\().*?(?=\))'

# # 匹配事件
Title = Jines.loc[(Jines['ParentId (more info)'].str.contains(pattern, regex=True, na=False))]

Attribute_temp = {"Affection": 0, "Stress": 0, "Darkness": 0}


Jines_file = r"/content/Jines.csv"  # 更改文件路径和扩展名

使用utf-8读取这个文件后，为我split成多行

In [68]:
def sanitize_filename(filename):
    invalid_chars = '<>:"/\\|?*\n'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

In [84]:
!rm -rf /content/events
!mkdir events


In [111]:
def set_json_value( a, key, value, verbose = True):
    if key in a and a[key] != value:
        event_name = a["Name_while_read_csv"]
        if verbose:
            print(f"Warning! Key {key} already exists in event {event_name}")
            print(f"try overwrite {a[key]} to {value}")
    else:
        a[key] = value
    return a

请为我实现一个python函数，输入和输出都是字符串
去掉字符串末尾3位可能的数字。

In [112]:
def remove_trailing_digits(s: str) -> str:
    """
    Remove up to three trailing digits from a string.

    :param s: Input string that may end with up to three digits.
    :return: String with the trailing digits removed.
    """
    # 初始化一个变量，用于计数尾部连续数字的数量
    trailing_digit_count = 0

    # 从字符串末尾开始，向前检查每个字符
    for char in reversed(s[-3:]):  # 查看最后三个字符
        if char.isdigit():  # 如果字符是数字
            trailing_digit_count += 1  # 增加计数
        else:
            break  # 如果遇到非数字字符，跳出循环

    # 如果尾部有数字，去除相应数量的字符
    if trailing_digit_count > 0:
        return s[:-trailing_digit_count]
    else:
        return s


# 示例用法
input_str = "example123"
output_str = remove_trailing_digits(input_str)
print(output_str)  # 应该输出 "example"


example


In [116]:
event_name_to_data = {}

# 把文件转换为txt
def format_output(row):

    # ID
    ParentId = f'{row["ParentId (more info)"]}'
    Category_temp = f'{row["Category"]}'
    Category = sanitize_filename(Category_temp)
    ID = f'{row["Id"]}'

    # 匹配标题
    regex1 = r"\w+(?= \()"
    title = re.search(regex1, ParentId)
    title_str = title.group()

    current_data = {}

    # 通过title_str 索引出数据
    if title_str in event_name_to_data:
        current_data = event_name_to_data[title_str]
    else:
        current_data = {"Name_while_read_csv":title_str,"options":[] }
        event_name_to_data[title_str] = current_data


    # 事件
    event_list = []

    # 匹配提问
    match = re.search(r"\(First Part\)", ParentId)
    match2 = re.search(r"\(First Part; end\)", ParentId)
    match3 = re.search(r"\(Third Part\)", ParentId)
    match4 = re.search(r"\(Second Part\)", ParentId)
    match5 = re.search(r"\(Fourth Part\)", ParentId)

    # 数值
    aff = f"Affection: {row['Affection']}"
    str = f"Stress: {row['Stress']}"
    dar = f"Darkness: {row['Darkness']}"

    # 匹配选项以及回复
    choose_time = re.search(r"\d+", ParentId)
    reply_ = re.search(r'(\(.*Option[0-9]+;end\))', ParentId)
    reply_2 = re.search(r'(\(.*Option[0-9]\))', ParentId)

    # 处理提问
    # if match or match2 or match4 or match5 or match3:
    if match or match2 or match3 or match4 or match5:

        Prefix = f'\n## 对话\n### Prefix Category_temp:{Category} ID:{ID}'

        current_data = set_json_value(current_data, "category", Category,False)

        if ID and len(ID) > 0 and ID != "nan":
            ID = remove_trailing_digits(ID)
            current_data = set_json_value(current_data, "id", ID, False)



        Ame = f"糖糖: {row['BodyCn']}"
        with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
            # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
            line = '\n'.join([Prefix, Ame])

            line_bytes = line.encode('utf-8')
            # 将字节对象写入到文件中
            line_str = line_bytes.decode('utf-8')
            # 将字符串对象写入到文件中
            f.write(line_str)

        return "\n".join([Prefix, Ame])

    # 处理选项
    elif row['Speaker/Action (in blue)'] == 'pi':
        # 跳过数值为空的回复
        try:
            key = f'\n### Option-{choose_time.group()}'
            user = f"User:　{row['BodyCn']}"

            if aff == 'Affection: nan':
                aff = ''
            if str == 'Stress: nan':
                str = ''
            if dar == 'Darkness: nan':
                dar = ''
            value = f"Attribute Change: {aff} {str} {dar}"

            if value == 'Attribute Change:   ':
                value = ''

            with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                line = '\n'.join([key, user, value])

                line_bytes = line.encode('utf-8')
                # 将字节对象写入到文件中
                line_str = line_bytes.decode('utf-8')
                # 将字符串对象写入到文件中
                f.write(line_str)
            return "\n".join([key, user, value])
        except:
            pass

    # 处理选项回复
    elif reply_ or (reply_2 and row['Speaker/Action (in blue)'] == 'ame'):
        try:
            key = f'\nReply：\n糖糖：{row["BodyCn"]}'

            if aff == 'Affection: nan':
                aff = ''
            if str == 'Stress: nan':
                str = ''
            if dar == 'Darkness: nan':
                dar = ''
            value = f"Attribute Change: {aff} {str} {dar}"

            if value == 'Attribute Change:   ':
                value = 'Attribute Change: None'

            if key == '\nReply：\n糖糖：nan':
                with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                    # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                    line = '\n'.join([value])
                    line_bytes = line.encode('utf-8')
                    # 将字节对象写入到文件中
                    line_str = line_bytes.decode('utf-8')
                    # 将字符串对象写入到文件中
                    f.write(line_str)

                return "\n".join([value])

            with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                line = '\n'.join([key, value])

                line_bytes = line.encode('utf-8')
                # 将字节对象写入到文件中
                line_str = line_bytes.decode('utf-8')
                # 将字符串对象写入到文件中
                f.write(line_str)

            return "\n".join([key, value])
        except:
            pass


In [115]:
# 转换为txt
re_Title = Title.apply(format_output, axis=1)
# output_str = re_Title.str.cat(sep="\n \n")  # 用空格分隔每个元素

Warning! Key category already exists in event StHi_FollowHi
try overwrite Random Noon Event_ Followers 100k+_and Stress Cap Raised to 120 to Random Noon Event_ Followers 250k+_and Stress Cap Raised to 120
Warning! Key category already exists in event StHi_FollowHi
try overwrite Random Noon Event_ Followers 100k+_and Stress Cap Raised to 120 to Random Noon Event_ Followers 250k+_and Stress Cap Raised to 120
Warning! Key category already exists in event StHi_FollowHi
try overwrite Random Noon Event_ Followers 100k+_and Stress Cap Raised to 120 to Random Noon Event_ Followers 250k+_and Stress Cap Raised to 120
Warning! Key category already exists in event StHi_FollowHi
try overwrite Random Noon Event_ Followers 100k+_and Stress Cap Raised to 120 to Random Noon Event_ Followers 250k+_and Stress Cap Raised to 120
Warning! Key category already exists in event StHi_FollowHi
try overwrite Random Noon Event_ Followers 100k+_and Stress Cap Raised to 120 to Random Noon Event_ Followers 500k+_and 

In [30]:
print(re_Title.head)

Empty DataFrame
Columns: [{"payload":{"allShortcutsEnabled":false, fileTree:{"data":{"items":[{"name":"Jines.csv", path:"data/Jines.csv", contentType:"file"}, {"name":"Jines.xlsx", path:"data/Jines.xlsx", contentType:"file"}.1, {"name":"emoji_story_23.jsonl", path:"data/emoji_story_23.jsonl", contentType:"file"}.2, {"name":"original_story_23.jsonl", path:"data/original_story_23.jsonl", contentType:"file"}], totalCount:4}, :{"items":[{"name":"data", path:"data", contentType:"directory"}, {"name":"notebook", path:"notebook", contentType:"directory"}.1, {"name":"src", path:"src", contentType:"directory"}.2, {"name":".DS_Store", path:".DS_Store", contentType:"file"}.3, {"name":".gitignore", path:".gitignore", contentType:"file"}.4, {"name":"LICENSE", path:"LICENSE", contentType:"file"}.5, {"name":"README.md", path:"README.md", contentType:"file"}].1, totalCount:7}}, fileTreeProcessingTime:5.894192, foldersToFetch:[], reducedMotionEnabled:null, repo:{"id":713164097, defaultBranch:"main", na

In [ ]:
# txt文件转换为jsonl
def parse_to_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = iter(f.readlines())
    dialogs = []
    dialog = {}
    option = {}
    for line in lines:
        line = line.strip()
        if line.startswith("## 对话") or line.startswith("## 对话组"):
            if dialog and option:
                dialog["options"].append(option)
                option = {}
            if dialog:
                dialogs.append(dialog)
            dialog = {"prefix": "", "options": []}
        elif line.startswith("### Prefix") or line.startswith('**Prefix'):
            prefix = next(lines).strip()
            ids, categories = search_in_excel(prefix)
            # print(ids, categories)
            if ids and categories:
                dialog["id"] = ids[0]
                dialog["category"] = categories[0]
            dialog["prefix"] = prefix
        elif line.startswith("### Option") or line.startswith('**Option'):
            if option:
                dialog["options"].append(option)
            option = {"user": "", "reply": "", "attribute_change": ""}

        elif line.startswith("User") or line.startswith("User:"):
            option["user"] = line[5:].strip()
        elif line.startswith("Reply") or line.startswith('**Reply:**'):
            option["reply"] = next(lines).strip()
        elif line.startswith("Attribute Change") or line.startswith('**Attribute Change:**'):
            option["attribute_change"] = line[17:].strip()

    if option:
        dialog["options"].append(option)
    if dialog:
        dialogs.append(dialog)

    with open('emoji_story_23.jsonl', 'a+', encoding="utf-8") as outfile:
        for entry in dialogs:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

In [ ]:
# 转换为jsonl
for filename in os.listdir('events'):
    if filename.endswith(".txt"):
        try:
            parse_to_jsonl(f'events/{filename}')
        except:
            shutil.move(f'move/{filename}', f'error/{filename}')
            print(filename)